# 트랜스포머 (Transformer)

* 참고: https://wikidocs.net/31379

* attention mechanism은 seq2seq의 입력 시퀀스 정보 손실을 보정해주기 위해 사용됨
* attention mechanism을 보정 목적이 아닌, 인코더와 디코더로 구성한 모델이 바로 트랜스포머
* 트랜스포머는 RNN을 사용하지 않고 인코더와 디코더를 설계하였으며, 성능도 RNN보다 우수함



## 포지셔널 인코딩

* 기존의 RNN은 단어의 위치를 따라 순차적으로 입력받아 단어의 위치정보를 활용할 수 있었음
* 트랜스포머의 경우, RNN을 활용하지 않았기 때문에 단어의 위치정보를 다른 방식으로 줄 필요가 있음
* 이를 위해 **각 단어의 임베딩 벡터에 위치 정보들을 더하게 되는데** 이를 포지셔널 인코딩이라 함
* 보통 포지셔널 인코딩은 sin, cos을 이용하여 계산

## 레이어 정규화

*  레이어 정규화에서는 텐서의 마지막 차원에 대해 평균과 분산을 구하고, 이 값을 통해 값을 정규화함
*  해당 정규화를 각 층의 연결에 편리하게 적용하기 위해 함수화한 `sublayer_connection()`을 선언

## 어텐션



*   트랜스포머 모델의 핵심이 되는 부분
*   트랜스포머에서는 multi-head attention과 self attention이라는 개념을 사용
  1.   multi-head attention
      * 디코더가 가지는 차원을 나누어 병렬로 어텐션을 진행
      *  마지막엔 병렬로 각 진행해 얻은 어텐션 헤드를 모두 연결
      * 이로 인해 다양한 시각에서 정보를 수집할 수 있는 효과를 얻음
  2.   self attention
      *   일반적인 어텐션의 경우, 특정 시점의 디코더 은닉상태와 모든 시점의 인코더 은닉상태를 활용
      *   이는 입력 문장과 다른 문장에 존재하는 단어간의 어텐션을 의미함
      *   반면 self attention은 은닉 상태를 동일하게 하여 어텐션을 진행
      *   이는 입력 문장 내 단어간의 어텐션을 의미함




*   트랜스포머 제안 논문에서는 scaled-dot product attention을 활용해 모델을 작성함



### scaled-dot product attention 구현

* scaled-dot product attention은 앞서 학습한 dot product attention과 거의 유사함
* 단 attention을 진행할 때 어텐션 스코어를 계산할 때 내적 값을 정규화
* 트랜스포머에서는 정규화할 때 K 벡터(=디코더 셀의 은닉 상태)의 차원을 루트를 취한 값을 사용

### multi-head attention 구현

* multi-head attention의 구현 과정
  1. query, key, value에 해당하는 값을 받고, 해당 값에 해당하는 행렬 생성
  2. 생성된 행렬들을 heads에 해당하는 수만큼 분리
  3. 분리한 행렬들에 대해 각각 어텐션을 수행
  4. 각 어텐션 결과들을 연결해 최종 어텐션 결과 생성





## 포지션-와이즈 피드 포워드 신경망



*   multi-head attention의 결과인 행렬을 입력받아 연산
*   일반적인 완전 연결 신경망(Dense layer)를 사용
*   position-wise FFNN은 인코더와 디코더에 모두 존재



## 인코더


* 인코더는 하나의 어텐션을 사용
  + encoder self-attention (multi-head self-attention과 동일)

## 디코더

* 디코더는 다음과 같은 구성의 반복으로 이루어짐
  1. masked decoder self-attention
  2. encoder-decoder attention
  3. position-wise FFNN

* 디코더에서는 2종류의 어텐션을 사용
  1.   masked decoder self-attention
    *   디코더에서는 인코더와는 달리 순차적으로 결과를 만들어 내야하기 때문에 다른 어텐션 방법을 사용함
    *   디코더 예측 시점 이후의 위치에 attention을 할 수 없도록 masking 처리
    *   결국 예측 시점에서 예측은 미리 알고 있는 위치까지만의 결과에 의존
  2.   encoder-decoder attention
    *   앞서 설명한 multi-head attention과 동일



## 트랜스포머를 활용한 챗봇

### konlpy 라이브러리

*    한글을 처리하기 위해 konlpy 라이브러리 설치

### 데이터 준비

* 처리에 필요한 각종 변수 선언
* filters에 해당되는 문자를 걸러주는 정규 표현식 컴파일



* 주소에서 데이터를 가져오는 `load_data()` 함수 선언



* 처리에 필요한 단어 사전을 생성하는 `load_vocab()` 함수 선언

* 문자열 데이터를 학습에 사용될 수 있도록 변현하는 `prepro_like_morphlized()` 함수 선언



* 단어 사전을 만들기 위해 단어들을 분리하는 `data_tokenizer()` 함수 선언

* encoder의 입력을 구성하기 위한 함수 `enc_processing()` 선언



* decoder의 입력을 구성하기 위한 함수 `dec_input_processing()` 선언

* decoder의 출력을 구성하기 위한 함수 `dec_target_processing()` 선언

* 모델에 데이터를 효율적으로 투입하도록 `train_input_fn()`, `eval_input_fn()` 함수 선언
* `rearrange()`는 dataset 객체가 데이터를 어떻게 변형시킬지 정의해둔 함수
* dataset.map은 rearrange 함수를 기반으로 데이터를 변형



* 모델의 예측은 배열로 생성되기 때문에 이를 확인하기 위해선 문자열로 변환이 필요
* 예측을 문자열로 변환해주는 `pred2string()` 함수 선언


* 챗봇 데이터 URL: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv
* 데이터 주소에서 데이터를 읽어들여 단어 사전과 사용 데이터 구성

### 모델 구성

* 앞서 작성한 트랜스포머 모델을 결합해 학습에 사용할 모델을 구성함

### 모델 학습

*   필요한 각종 인자들을 설정
*   인자에 따라 학습 결과가 달라질 수 있기 때문에 세심한 조정이 필요


*   앞서 선언한 processing 함수로 데이터를 모델에 투입할 수 있도록 가공
*   평가 데이터에도 동일하게 가공

* 앞서 선언한 함수를 통해 모델을 선언하고 학습
* `tf.estimator`를 사용해 간편하게 학습 모듈 구성


* 학습한 모델을 사용해 챗봇을 사용
* 예측 결과를 문자열로 변환할 때는 앞서 선언한 `pred2string()` 함수를 이용
* 입력에 대한 응답이 생성되는 것을 확인할 수 있음


### 예측

* 학습한 모델을 사용해 챗봇을 사용
* 예측 결과를 문자열로 변환할 때는 앞서 선언한 `pred2string()` 함수를 이용
* 입력에 대한 응답이 생성되는 것을 확인할 수 있음
